In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import time
import os
import glob
%matplotlib inline

In [2]:
train_df=pd.read_json(r'/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/train.json')

In [3]:
train_df.head(3)

,band_1,band_2,id,inc_angle,is_iceberg
0,"[-27.878360999999998, -27.15416, -28.668615, -...","[-27.154118, -29.537888, -31.0306, -32.190483,...",dfd5f913,43.9239,0
1,"[-12.242375, -14.920304999999999, -14.920363, ...","[-31.506321, -27.984554, -26.645678, -23.76760...",e25388fd,38.1562,0
2,"[-24.603676, -24.603714, -24.871029, -23.15277...","[-24.870956, -24.092632, -20.653963, -19.41104...",58b2aaa0,45.2859,1


In [4]:
def generate_minibatch(inputs=None,batch_size=128):
    data_cnt=inputs.shape[0]
    if batch_size>data_cnt:
        raise('batch bigger than data nmber')
    df=inputs.sample(data_cnt)
    start_idx=0
    end_idx=batch_size
    while True:
        if end_idx<start_idx:
            batch_df=df.iloc[start_idx:].append(df.iloc[:end_idx])
        else:
            batch_df=df.iloc[start_idx:end_idx]
        start_idx=(start_idx+batch_size)%data_cnt
        end_idx=(start_idx+batch_size)%data_cnt
        band1=np.array([x for x in batch_df['band_1']])
        band2=np.array([x for x in batch_df['band_2']])
        imgs=np.stack([band1.reshape(-1,75,75),band2.reshape(-1,75,75)],axis=-1)
        labels=batch_df['is_iceberg'].astype(np.int32).values
        yield imgs,labels

In [5]:
mini=generate_minibatch(inputs=train_df,batch_size=128)
img,label=next(mini)

#  构建网络

In [6]:
def inference(input_tensor,train,regularizer):
    with tf.variable_scope('layer1-conv1') as scope:
        conv1_weights=tf.get_variable(name='weight',shape=[5,5,2,32],initializer=tf.random_normal_initializer(stddev=0.1,mean=0,seed=9))
        conv1_biases=tf.get_variable(name='bias',shape=[32],initializer=tf.constant_initializer(0.1))
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    with tf.name_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable(name='weight',shape=[5,5,32,64],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0))
        conv2_biases=tf.get_variable(name='bias',shape=64,initializer=tf.constant_initializer(0.1))
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope('layer4-pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        
    with tf.variable_scope('layer5-conv3'):
        conv3_weights=tf.get_variable('weight',shape=[3,3,64,128],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0))
        conv3_biases=tf.get_variable('bias',shape=[128],initializer=tf.constant_initializer(0.0))
        conv3=tf.nn.conv2d(pool2,conv3_weights,strides=[1,1,1,1],padding='SAME')
        relu3=tf.nn.relu(tf.nn.bias_add(conv3,conv3_biases))
    with tf.name_scope('layer6-pool3'):
        pool3=tf.nn.max_pool(relu3,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
    
    with tf.variable_scope('layer7-conv4'):
        conv4_weights=tf.get_variable(name='weight',shape=[3,3,128,128],initializer=tf.truncated_normal_initializer(stddev=0.1,mean=0.0,seed=9))
        conv4_biases=tf.get_variable(name='bias',shape=[128],initializer=tf.constant_initializer(0.1))
        conv4=tf.nn.conv2d(pool3,conv4_weights,strides=[1,1,1,1],padding='SAME')
        relu4=tf.nn.relu(tf.nn.bias_add(conv4,conv4_biases))
    with tf.name_scope('layer8-pool4'):
        pool4=tf.nn.max_pool(relu4,ksize=[1,2,2,1],strides=[1,2,2,1],padding='VALID')
        nodes=4*4*128
        flatened=tf.reshape(pool4,[-1,nodes])
    
    with tf.variable_scope('layer9-fc1'):
        fc1_weights=tf.get_variable(name='weight',shape=[nodes,1024],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable(name='bias',shape=[1024],initializer=tf.constant_initializer(0.1))
        fc1=tf.nn.relu(tf.matmul(flatened,fc1_weights)+fc1_biases)
        if train:
            fc1=tf.nn.dropout(fc1,0.5)
    
    with tf.variable_scope('layer10-fc2'):
        fc2_weights=tf.get_variable(name='weight',shape=[1024,512],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable(name='bias',shape=[512],initializer=tf.constant_initializer(0.1))
        fc2=tf.nn.relu(tf.matmul(fc1,fc2_weights)+fc2_biases)
        if train:
            fc2=tf.nn.dropout(fc2,0.5)
    
    with tf.variable_scope('layer11-fc3'):
        fc3_weights=tf.get_variable(name='weight',shape=[512,2],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if regularizer!=None:
            tf.add_to_collection('losses',regularizer(fc3_weights))
        fc3_biases=tf.get_variable(name='bias',shape=[2],initializer=tf.constant_initializer(0.1))
        logit=tf.matmul(fc2,fc3_weights)+fc3_biases
    return logit

In [7]:
img_h=75
img_w=75
img_c=2
n_steps=1000
batch_size=64
model_path='/home/yw/study/Competition/Iceberg-Classifier-Challenge_20171125/experiments/20171125_CNN_model/cnn.ckpt'

In [8]:
conv_graph=tf.Graph()

In [10]:
with conv_graph.as_default():
    x=tf.placeholder(shape=[None,img_w,img_h,img_c],dtype=tf.float32,name='input_imgs')
    y=tf.placeholder(shape=[None],dtype=tf.int32,name='input_labels')
    dropout_flag=tf.placeholder(dtype=tf.bool,shape=None,name='droupout_flag')
    regularizer=tf.contrib.layers.l2_regularizer(0.0001)
    logits=inference(x,False,regularizer)
    b = tf.constant(value=1,dtype=tf.float32)
    logits_eval = tf.multiply(logits,b,name='logits_eval')
    
    loss=tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits,labels=y)
    avg_loss=tf.reduce_mean(loss)
    train_op=tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)
    saver=tf.train.Saver()

# 训练模型

In [ ]:
with tf.Session(graph=conv_graph) as sess:
    start_time=time.clock()
    sess.run(tf.global_variables_initializer())
    train_loss=0.0
    i=0
    for img,label in generate_minibatch(train_df,batch_size):
        sess.run(train_op,feed_dict={x:img,y:label,dropout_flag:True})
        if i%100==0:
            train_loss=sess.run(avg_loss,feed_dict={x:img,y:label,dropout_flag:False})
            print('train_loss:',train_loss)
        i+=1
    end_time=time.clock()
    print('Time:',end_time-start_time)
    saver.save(sess,model_path)

train_loss: 8110.55
train_loss: 0.905599
train_loss: 0.482073
train_loss: 0.384247
train_loss: 0.14674
train_loss: 0.137088
train_loss: 0.177965
train_loss: 0.100015
train_loss: 0.0993202
